In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time


In [ ]:
!pip install konlpy

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/kor.txt') as f :
  lines = f.readlines()

In [ ]:
targ, inp, trash = zip(*[line.split('\t') for line in lines])

In [ ]:
inp[-1]

'의심의 여지 없이 세상에는 어떤 남자이든 정확히 딱 알맞는 여자와 결혼하거나 그 반대의 상황이 존재하지. 그런데 인간이 수백 명의 사람만 알고 지내는 사이가 될 기회를 갖는다고 생각해 보면, 또 그 수백 명 중 열여 명 쯤 이하만 잘 알 수 있고, 그리고 나서 그 열여 명 중에 한두 명만 친구가 될 수 있다면, 그리고 또 만일 우리가 이 세상에 살고 있는 수백만 명의 사람들만 기억하고 있다면, 딱 맞는 남자는 지구가 생겨난 이래로 딱 맞는 여자를 단 한번도 만난 적이 없을 수도 있을 거라는 사실을 쉽게 눈치챌 수 있을 거야.'

In [ ]:
from konlpy.tag import Hannanum

hannanum = Hannanum()

hannanum.morphs(inp[-1])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
word_tokenize(targ[-1].lower())

In [ ]:
x_tokens = [hannanum.morphs(x) for x in inp]
y_tokens = [word_tokenize(x.lower()) for x in targ]

In [ ]:
def text_encoding(lines):

    vocab, index = {}, 3  # start indexing from 1

    vocab['<pad>'] = 0  # add a padding token

    vocab['<bos>'] = 1  # begin of sentence

    vocab['<eos>'] = 2  # end of sentence

    preprocessed_tokens = []

    maxlen = -1

    for sentence in lines:

        for token in sentence:

            if token not in vocab:

                vocab[token] = index

                index += 1

        if maxlen < len(sentence):

            maxlen = len(sentence)

    arr = np.zeros((len(lines), maxlen+2), dtype='int32')

    for i, sentence in enumerate(lines):

        for j, token in enumerate(sentence):

            arr[i, j+1] = vocab[token]

        arr[i, 0] = vocab['<bos>']

        arr[i, len(sentence)+1] = vocab['<eos>']

    return arr, vocab


In [ ]:
#그리디 서치에 버그가 있다.

In [ ]:
x_train,x_vocab = text_encoding(x_tokens)
y_train,y_vocab = text_encoding(y_tokens)

In [ ]:
inverse_x_vocab = {index : token for token,index in x_vocab.items()}
inverse_y_vocab = {index : token for token,index in y_vocab.items()}

In [ ]:
def text_decoding(line, vo):
  return [vo[x] for x in line]

In [ ]:
print(text_decoding(x_train[-1],inverse_x_vocab))

['<bos>', '의심', '의', '여', '이', '지', '없이', '세상', '에는', '어떤', '남자', '이', '든', '정확히', '딱', '알맞', '는', '여자', '와', '결혼', '하', '거나', '그', '반대', '의', '상황', '이', '존재', '하', '지', '.', '그런데', '인간', '이', '수백', '명', '의', '사람', '만', '알', '고', '지내', '는', '사이', '가', '되', 'ㄹ', '기회', '를', '갖', '는다', '고', '생각', '하', '어', '보', '면', ',', '또', '그', '수백', '명', '중', '열', '여', '명', '쯤', '이하', '만', '잘', '알', 'ㄹ', '수', '있', '고', ',', '그리고', '나', '서', '그', '열', '여', '명', '중', '에', '한두', '명', '만', '친구', '가', '되', 'ㄹ', '수', '있', '다면', ',', '그리고', '또', '만', '이', 'ㄹ', '우리', '가', '이', '세상', '에', '살', '고', '있', '는', '수백만', '명', '의', '사람들', '만', '기억', '하고', '있', '다면', ',', '딱', '맞', '는', '남자', '는', '지구', '가', '생기', '어', '나', 'ㄴ', '이래', '로', '딱', '맞', '는', '여자', '를', '달', 'ㄴ', '한번', '도', '만나', 'ㄴ', '적', '이', '없', '을', '수', '도', '있', '을', '것', '이', '라는', '사실', '을', '쉽', '게', '눈치채', 'ㄹ', '수', '있', '을', '것', '이', '야', '.', '<eos>']


In [ ]:
x_train.shape

(5870, 169)

In [ ]:
np.savez('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/kor-eng',x_train = x_train, y_train= y_train)

In [ ]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/krvocab.json','w') as f :
  json.dump(x_vocab,f)
with open('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/envocab.json','w') as f :
  json.dump(y_vocab,f)

In [ ]:
npzfile = np.load('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/kor-eng.npz')
x_train = npzfile['x_train']
y_train = npzfile['y_train']

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/krvocab.json','rb') as f :
  x_vocab = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/실전딥러닝2023/envocab.json','rb') as f :
  y_vocab = json.load(f)

In [ ]:
inverse_x_vocab = {index : token for token,index in x_vocab.items()}
inverse_y_vocab = {index : token for token,index in y_vocab.items()}

In [ ]:
buffer_size = len(x_train)
batch_size = 16
embedding_dim = 1024
latent_dim = 1024
x_vocab_size = len(x_vocab)
y_vocab_size = len(y_vocab)
dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(buffer_size)
dataset =dataset.batch(batch_size)

In [ ]:
for example_input_batch,example_target_batch in dataset.take(1):
  break

In [ ]:
example_input_batch = example_input_batch[:,:np.max(np.count_nonzero(example_input_batch,axis =1))]
example_target_batch = example_target_batch[:,:np.max(np.count_nonzero(example_target_batch,axis =1))]

In [ ]:
example_input_batch.shape

TensorShape([16, 23])

In [ ]:
def build_encoder():
  input_seq = tf.keras.layers.Input((None,))
  x= tf.keras.layers.Embedding(x_vocab_size, embedding_dim)(input_seq)
  x,s1 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru1',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  x,s2 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru2',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  x,s3 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru3',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  model = tf.keras.Model(input_seq,[x,s1,s2,s3])
  return model

In [ ]:
encoder = build_encoder()

In [ ]:
example_input_batch.shape

TensorShape([16, 23])

In [ ]:
last_output, last_state1, last_state2, last_state3 = encoder(example_input_batch)

In [ ]:
last_output.shape

TensorShape([16, 23, 1024])

In [ ]:
def build_decoder():
  target_seq = tf.keras.layers.Input((None,))
  s1 = tf.keras.layers.Input((latent_dim,))
  s2 = tf.keras.layers.Input((latent_dim,))
  s3 = tf.keras.layers.Input((latent_dim,))

  embedding = tf.keras.layers.Embedding(y_vocab_size, embedding_dim)(target_seq)
  x,output_s1 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru1',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(embedding,initial_state =s1,mask=tf.cast(target_seq !=0,bool))

  x,output_s2 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru2',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,initial_state =s2,mask=tf.cast(target_seq !=0,bool))
  x,output_s3 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru3',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,initial_state =s3,mask=tf.cast(target_seq !=0,bool))
  logits = tf.keras.layers.Dense(y_vocab_size)(x)
  model = tf.keras.Model([target_seq,s1,s2,s3],[logits,output_s1,output_s2,output_s3])
  return model

In [ ]:
decoder = build_decoder()

In [ ]:
logits ,s1,s2,s3 =decoder([example_target_batch,last_state1,last_state2,last_state3])

In [ ]:
logits.shape

TensorShape([16, 20, 3141])

In [ ]:
s1.shape

TensorShape([16, 1024])

In [ ]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits =True, reduction ='none')
def batch_loss(y_true, y_pred):
  loss = loss_func(y_true,y_pred)
  mask = tf.cast(y_true != 0, tf.float32)
  loss = loss *mask
  return tf.reduce_sum(loss)/tf.reduce_sum(mask)


In [ ]:
batch_loss(example_target_batch[:,1:],logits[:,:-1,:])

<tf.Tensor: shape=(), dtype=float32, numpy=8.052269>

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)
def predict(x,y,training=True):
  _,ls1,ls2,ls3 = encoder(x,training =training)
  logits, _,_,_ = decoder([y,ls1,ls2,ls3],training = training)
  return logits

In [ ]:
def train_step(x,y,t_v):
  x= x[:,:tf.reduce_max(tf.math.count_nonzero(x,axis=1))]
  y= y[:,:tf.reduce_max(tf.math.count_nonzero(y,axis=1))]
  with tf.GradientTape() as tape:
    logits  =predict(x,y)
    loss = batch_loss(y[:,1:],logits[:,:-1,:])
  gradients = tape.gradient(loss,t_v)
  optimizer.apply_gradients(zip(gradients, t_v))
  return loss

In [ ]:
t_v = encoder.trainable_variables +decoder.trainable_variables

for epoch in range(50):
  start = time.time()

  loss_sum = 0
  for x_batch,y_batch in dataset:
    loss = train_step(x_batch,y_batch,t_v)
    loss_sum += loss
  print('time for epoch{}is {} sec: training_loss = {}'.format(epoch+1,time.time()-start,loss))

time for epoch1is 92.94394588470459 sec: training_loss = 4.2454447746276855
time for epoch2is 64.71694159507751 sec: training_loss = 4.058753967285156
time for epoch3is 60.93182349205017 sec: training_loss = 3.6805193424224854
time for epoch4is 63.69733119010925 sec: training_loss = 3.3260550498962402
time for epoch5is 81.90356755256653 sec: training_loss = 2.8478100299835205
time for epoch6is 60.30056381225586 sec: training_loss = 3.32228946685791
time for epoch7is 81.90387010574341 sec: training_loss = 2.4085617065429688
time for epoch8is 61.744802474975586 sec: training_loss = 2.4996538162231445
time for epoch9is 81.90269041061401 sec: training_loss = 2.277601957321167
time for epoch10is 60.33395767211914 sec: training_loss = 1.9484912157058716
time for epoch11is 81.90353059768677 sec: training_loss = 1.8914361000061035
time for epoch12is 59.349772930145264 sec: training_loss = 1.9507839679718018
time for epoch13is 81.90364956855774 sec: training_loss = 1.6822960376739502
time for e

KeyboardInterrupt: ignored

In [ ]:
!pip3 install pyyaml h5py

In [ ]:
def translate(src,max_steps = 100):
  src_tokens = np.array([x_vocab[x] for x in hannanum.morphs(src)])
  src_tokens = np.insert(src_tokens,0,[1])
  src_tokens = np.append(src_tokens,2)
  x_test = tf.expand_dims(src_tokens,axis =0)
  _,s1,s2,s3 = encoder(x_test,training = False)

  y_test = tf.expand_dims(tf.constant([y_vocab['<bos>']]),axis = 0)
  output_seq = []
  for _ in range(max_steps):
    logits,s1,s2,s3 = decoder([y_test,s1,s2,s3],training = False)
    y_test = tf.argmax(logits,axis =2)
    #print(y_test)
    pred = tf.squeeze(y_test,axis =0)
    #print(pred)

    if pred == y_vocab['<eos>']:
      #print("zz")
      break
    output_seq.append(pred.numpy())
  return ' '.join([inverse_y_vocab[x[0]] for x in output_seq])

In [ ]:
translate('잘 안 되네')

"do n't be so silly ."

In [ ]:
def beam_translate(src,max_steps = 100,k = 16):
  src_tokens = np.array([x_vocab[x] for x in hannanum.morphs(src)])
  src_tokens = np.insert(src_tokens,0,[1])
  src_tokens = np.append(src_tokens,2)
  x_test = tf.expand_dims(src_tokens,axis =0)
  _,s1,s2,s3 = encoder(x_test,training = False)

  last_token = tf.constant([y_vocab['<bos>']])
  candidates = [(0.,tf.expand_dims(last_token,axis = 0),s1,s2,s3,[int(last_token)],False)]

  for _ in range(max_steps):
    new_candidates = []
    for score , token,s1,s2,s3,output_seq,eos in candidates :
      if eos:
        new_candidates.append((score,token,s1,s2,s3,output_seq,eos))
        continue
      logits,s1,s2,s3 = decoder([token,s1,s2,s3],training = False)
      probs = tf.nn.log_softmax(logits,axis =2)
      values, indices = tf.math.top_k(tf.squeeze(probs),k=k)
      for prob, idx in zip(values, indices):
        eos = (int(idx)==y_vocab['<eos>'])
        last_token = tf.constant([int(idx)])
        new_candidates.append(
            (score +float(prob),tf.expand_dims(last_token,axis= 0),s1,s2,s3,
             output_seq + [int(last_token)],eos)
        )
    candidates = sorted(new_candidates, key = lambda t: -t[0])[:k]
  return [(candidates[i][0],' '.join([inverse_y_vocab[x] for x in candidates[i][5]])) for i in range(k)]

In [ ]:
beam_translate('잘 안되네')

[(-4.257024133454252, "<bos> it 's not a liar . <eos>"),
 (-4.706906879146118, "<bos> do n't you like it ? <eos>"),
 (-4.754112625028938, "<bos> do n't you laugh ? <eos>"),
 (-4.8033481109305285, "<bos> you 're mine . <eos>"),
 (-4.8747765948064625, "<bos> do n't you ? <eos>"),
 (-5.192608612705953, "<bos> do n't you like me ? <eos>"),
 (-5.2122524314327165, "<bos> do n't you leave ? <eos>"),
 (-5.272478144150227, "<bos> do n't you do that ? <eos>"),
 (-5.3478018215391785, "<bos> you 're lonely . <eos>"),
 (-5.731973258632934, "<bos> it 's not a fault . <eos>"),
 (-5.853176517717657, "<bos> do n't you do that . <eos>"),
 (-6.110417670832248, "<bos> do n't you help me ? <eos>"),
 (-6.424333076298353, "<bos> do n't you do it ? <eos>"),
 (-7.164485688488639, "<bos> do n't you like a passport . <eos>"),
 (-7.188707823494042, "<bos> do n't you like a passport ? <eos>"),
 (-7.2123451717197895, "<bos> do n't you like a cell phone . <eos>")]

In [ ]:
위의 코드를 통해 autoencoder 모델을 훈련시켰다.
i want to show the output sequence with the test images(example_input_batch[0])
Use greedy search for prediction

밑의 코드를 조금 수정해서 beam_translate function을 정의해줘.

beam_translate(tf.expand_dims(example_input_batch[0], axis=0))를 실행하면 output sequence가 출력되도록 해줘 .
example_input_batch[0].shape is  TensorShape([28, 28, 1])

In [ ]:

밑의 코드들을 수정해서 image를 넣으면 output seq가 나오는 모델을 학습하는 코드 알려줘
형식만  밑의 코드를 따라가고 세부 설정은 밑의 설명에 따라서 수정해줘


input_imgs.shape is (4434, 28, 28, 1)

output_seqs.shape is  (4434, 202)
Encoder is a CNN that outputs a 10-dim latent vector with
an input image and Decoder is 3 layers of LSTMs


def build_encoder():
  input_seq = tf.keras.layers.Input((None,))
  x= tf.keras.layers.Embedding(x_vocab_size, embedding_dim)(input_seq)
  x,s1 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru1',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  x,s2 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru2',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  x,s3 = tf.keras.layers.GRU(latent_dim,name = 'encoder_gru3',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,mask=tf.cast(input_seq !=0,bool))
  model = tf.keras.Model(input_seq,[x,s1,s2,s3])
  return model
encoder = build_encoder()

def build_decoder():
  target_seq = tf.keras.layers.Input((None,))
  s1 = tf.keras.layers.Input((latent_dim,))
  s2 = tf.keras.layers.Input((latent_dim,))
  s3 = tf.keras.layers.Input((latent_dim,))

  embedding = tf.keras.layers.Embedding(y_vocab_size, embedding_dim)(target_seq)
  x,output_s1 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru1',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(embedding,initial_state =s1,mask=tf.cast(target_seq !=0,bool))

  x,output_s2 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru2',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,initial_state =s2,mask=tf.cast(target_seq !=0,bool))
  x,output_s3 = tf.keras.layers.GRU(latent_dim,name = 'decoder_gru3',
                             return_sequences = True,
                             return_state = True,
                             recurrent_initializer='glorot_uniform')(x,initial_state =s3,mask=tf.cast(target_seq !=0,bool))
  logits = tf.keras.layers.Dense(y_vocab_size)(x)
  model = tf.keras.Model([target_seq,s1,s2,s3],[logits,output_s1,output_s2,output_s3])
  return model

decoder = build_decoder()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits =True, reduction ='none')
def batch_loss(y_true, y_pred):
  loss = loss_func(y_true,y_pred)
  mask = tf.cast(y_true != 0, tf.float32)
  loss = loss *mask
  return tf.reduce_sum(loss)/tf.reduce_sum(mask)

optimizer = tf.keras.optimizers.Adam(1e-4)
def predict(x,y,training=True):
  _,ls1,ls2,ls3 = encoder(x,training =training)
  logits, _,_,_ = decoder([y,ls1,ls2,ls3],training = training)
  return logits

def train_step(x,y,t_v):
  x= x[:,:tf.reduce_max(tf.math.count_nonzero(x,axis=1))]
  y= y[:,:tf.reduce_max(tf.math.count_nonzero(y,axis=1))]
  with tf.GradientTape() as tape:
    logits  =predict(x,y)
    loss = batch_loss(y[:,1:],logits[:,:-1,:])
  gradients = tape.gradient(loss,t_v)
  optimizer.apply_gradients(zip(gradients, t_v))
  return loss

t_v = encoder.trainable_variables +decoder.trainable_variables

for epoch in range(50):
  start = time.time()

  loss_sum = 0
  for x_batch,y_batch in dataset:
    loss = train_step(x_batch,y_batch,t_v)
    loss_sum += loss
  print('time for epoch{}is {} sec: training_loss = {}'.format(epoch+1,time.time()-start,loss))